# Project 2 Code Part 1 (Data Prepping and sentiment score assignment)
### Name: Arata Fujii

In [2]:
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from sklearn.linear_model import LogisticRegression
from transformers import pipeline

# Data Prepping

In [3]:
#loading data
folder_path = '/Users/aratafujii/Desktop/Project2'
df_susb = pd.read_csv(os.path.join(folder_path,"SUSBprices.csv"))
df_esgb = pd.read_csv(os.path.join(folder_path,"ESGBprices.csv"))

In [4]:
df_news = pd.read_csv(os.path.join(folder_path,"envnews.csv"))
df_news.count()

Title             29111
Intro Text        29977
Authors           25489
Article Text      29691
Date Published    27618
dtype: int64

In [5]:
df_esgb = df_esgb.sort_values(by='Date')
df_susb = df_susb.sort_values(by='Date')

# calculate 1-week (7 days) backward and forward average for bond prices
df_esgb['ESGBBackAvg'] = df_esgb['Adj Close'].rolling(window=7, min_periods=1).mean()
df_susb['SUSBBackAvg'] = df_susb['Adj Close'].rolling(window=7, min_periods=1).mean()
df_esgb['ESGBForwAvg'] = df_esgb['Adj Close'].rolling(window=7, min_periods=1).mean().shift(-6)
df_susb['SUSBForwAvg'] = df_susb['Adj Close'].rolling(window=7, min_periods=1).mean().shift(-6)

#get difference between the two prices
df_esgb['ESGBDifference'] = df_esgb['ESGBForwAvg'] - df_esgb['ESGBBackAvg']
df_susb['SUSBDifference'] = df_susb['SUSBForwAvg'] - df_susb['SUSBBackAvg']

df_esgb['ESGBClose'] = df_esgb['Adj Close']
df_susb['SUSBClose'] = df_susb['Adj Close']
df_esgb['ESGBVolume'] = df_esgb['Volume']
df_susb['SUSBVolume'] = df_esgb['Volume']

df_esgb_new = df_esgb[['Date', 'ESGBDifference', 'ESGBClose', 'ESGBVolume']]
df_susb_new = df_susb[['Date', 'SUSBDifference', 'SUSBClose']]

df_merged = pd.merge(df_esgb_new, df_susb_new, on='Date', suffixes=('_esgb', '_susb'))

df_merged.head(30)

,Date,ESGBDifference,ESGBClose,ESGBVolume,SUSBDifference,SUSBClose
0,2021-06-29,0.097695,22.599146,800100,3.156871e-02,24.471588
1,2021-06-30,0.102663,22.628958,100,3.331886e-02,24.476286
2,2021-07-01,0.112363,22.629864,100,3.802371e-02,24.464993
3,2021-07-02,0.104084,22.670511,100,3.224700e-02,24.497927
4,2021-07-06,0.089230,22.741880,10200,2.579914e-02,24.507324
5,2021-07-07,0.071885,22.801504,600,1.733357e-02,24.526144
6,2021-07-08,0.051752,22.806025,10000,4.166143e-03,24.516735
7,2021-07-09,0.040524,22.738266,0,-1.611714e-03,24.512033
8,2021-07-12,0.042590,22.733753,10000,-6.711429e-04,24.497927
9,2021-07-13,0.048526,22.661484,900,-5.714286e-07,24.469696


In [6]:
# calculate 2-week (14 days) backward and forward average for bond prices
df_esgb['ESGBBackAvg_14'] = df_esgb['Adj Close'].rolling(window=14, min_periods=1).mean()
df_susb['SUSBBackAvg_14'] = df_susb['Adj Close'].rolling(window=14, min_periods=1).mean()
df_esgb['ESGBForwAvg_14'] = df_esgb['Adj Close'].rolling(window=14, min_periods=1).mean().shift(-13)
df_susb['SUSBForwAvg_14'] = df_susb['Adj Close'].rolling(window=14, min_periods=1).mean().shift(-13)

#calculate the differences between the forward and backward averages
df_esgb['ESGBDifference_14'] = df_esgb['ESGBForwAvg_14'] - df_esgb['ESGBBackAvg_14']
df_susb['SUSBDifference_14'] = df_susb['SUSBForwAvg_14'] - df_susb['SUSBBackAvg_14']

df_esgb['ESGBClose_14'] = df_esgb['Adj Close']
df_susb['SUSBClose_14'] = df_susb['Adj Close']
df_esgb['ESGBVolume_14'] = df_esgb['Volume']
df_susb['SUSBVolume_14'] = df_susb['Volume']

df_esgb_new = df_esgb[['Date', 'ESGBDifference_14', 'ESGBClose_14']]
df_susb_new = df_susb[['Date', 'SUSBDifference_14', 'SUSBClose_14']]

df_merged_14 = pd.merge(df_esgb_new, df_susb_new, on='Date', suffixes=('_esgb', '_susb'))
df_merged = pd.merge(df_merged, df_merged_14, on='Date', suffixes=('_7', '_14'))

df_merged.head(30)

,Date,ESGBDifference,ESGBClose,ESGBVolume,SUSBDifference,SUSBClose,ESGBDifference_14,ESGBClose_14,SUSBDifference_14,SUSBClose_14
0,2021-06-29,0.097695,22.599146,800100,3.156871e-02,24.471588,0.127894,22.599146,0.014517,24.471588
1,2021-06-30,0.102663,22.628958,100,3.331886e-02,24.476286,0.131444,22.628958,0.024122,24.476286
2,2021-07-01,0.112363,22.629864,100,3.802371e-02,24.464993,0.138885,22.629864,0.031846,24.464993
3,2021-07-02,0.104084,22.670511,100,3.224700e-02,24.497927,0.140284,22.670511,0.039369,24.497927
4,2021-07-06,0.089230,22.741880,10200,2.579914e-02,24.507324,0.130592,22.741880,0.041852,24.507324
5,2021-07-07,0.071885,22.801504,600,1.733357e-02,24.526144,0.112538,22.801504,0.039299,24.526144
6,2021-07-08,0.051752,22.806025,10000,4.166143e-03,24.516735,0.099180,22.806025,0.035067,24.516735
7,2021-07-09,0.040524,22.738266,0,-1.611714e-03,24.512033,0.099164,22.738266,0.031844,24.512033
8,2021-07-12,0.042590,22.733753,10000,-6.711429e-04,24.497927,0.104672,22.733753,0.029965,24.497927
9,2021-07-13,0.048526,22.661484,900,-5.714286e-07,24.469696,0.120242,22.661484,0.034467,24.469696


In [7]:
#change to datetime
df_news['Date Published'] = pd.to_datetime(df_news['Date Published'])
df_esgb['Date'] = pd.to_datetime(df_esgb['Date'])
df_susb['Date'] = pd.to_datetime(df_susb['Date'])

# merge df_news with df_esgb to add the ESGB bond prices
df_news = df_news.merge(df_esgb, left_on='Date Published', right_on='Date', how='left', suffixes=('', '_esgb'))
df_news.rename(columns={'Adj Close': 'ESGB'}, inplace=True)
df_news.count()

df_news.drop(['Date'], axis=1, inplace=True)

#merge the other aswell
df_news = df_news.merge(df_susb, left_on='Date Published', right_on='Date', how='left', suffixes=('', '_susb'))
df_news.rename(columns={'Adj Close': 'SUSB'}, inplace=True)

df_news.drop(['Date'], axis=1, inplace=True)
df_news['Date'] = df_news['Date Published']
df_news.drop('Date Published', axis = 1, inplace=True)


#useing minmaxscaler to preprocess
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
columns_to_scale = ['SUSBClose', 'ESGBClose']
df_news[columns_to_scale] = scaler.fit_transform(df_news[columns_to_scale])

# Assign sentiment scores to articles

In [8]:
#take care of NaN values
df_news = df_news.dropna(subset=['Article Text', 'Date', 'ESGBDifference','SUSBDifference'])

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# taken from hugging face Official
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

#running 3 models on article text to see which one works best if any
model_list = ["cardiffnlp/twitter-roberta-base-sentiment-latest", "nlptown/bert-base-multilingual-uncased-sentiment", "hakonmh/sentiment-xdistil-uncased"]

def get_sentiment_score(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
    output = pretrained(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return np.max(scores) 

for model in model_list:
    pretrained = AutoModelForSequenceClassification.from_pretrained(model)
    tokenizer = AutoTokenizer.from_pretrained(model)
    df_news['Sentiment Score' + model] = df_news["Article Text"].apply(get_sentiment_score)
    
df_news.head()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,Title,Intro Text,Authors,Article Text,Open,High,Low,Close,ESGB,Volume,...,SUSBVolume,SUSBBackAvg_14,SUSBForwAvg_14,SUSBDifference_14,SUSBClose_14,SUSBVolume_14,Date,Sentiment Scorecardiffnlp/twitter-roberta-base-sentiment-latest,Sentiment Scorenlptown/bert-base-multilingual-uncased-sentiment,Sentiment Scorehakonmh/sentiment-xdistil-uncased
0,Liz Truss ‘will approve more oil drilling if ...,Tory leadership candidate criticised by campai...,"['Rob Davies', '@ByRobDavies']",Liz Truss will sign off on a push for more oil...,21.813000,21.813000,21.813000,21.813000,20.171234,0.0,...,NaN,23.110091,22.889202,-0.220888,22.994223,114600.0,2022-08-30,0.755918,0.315693,0.869581
2,Visiting green spaces deters mental health dr...,Positive effects were stronger among those rep...,"['Damien Gayle', '@damiengayle']","Visits to parks, community gardens and other u...",21.299999,21.320000,21.299999,21.309999,20.028095,4000.0,...,NaN,23.013015,23.240937,0.227922,23.165962,194600.0,2023-01-17,0.828758,0.403952,0.481355
3,Bought too much red cabbage? Turn it into a fe...,This fantastic vegan centrepiece makes full us...,['Tom Hunt'],"I devised today’s nut roast for Oddbox, a veg ...",21.110001,21.110001,21.084999,21.084999,20.707699,2400.0,...,NaN,24.064723,24.244775,0.180052,24.224943,89900.0,2023-12-22,0.721949,0.599755,0.997967
4,‘This year has been very good’: readers’ UK bu...,Readers share their favourite sightings over t...,['Guardian readers'],‘Constant companions to our gardening’A peacoc...,21.000000,21.000000,21.000000,21.000000,20.624222,100.0,...,NaN,23.998831,24.218568,0.219737,24.145615,76700.0,2023-12-19,0.522691,0.560167,0.989446
5,"Country diary: Willow tits are here, but good ...","Tittesworth Reservoir, Staffordshire: It’s eas...",['Mark Cocker'],The numerous bird feeders here are permanently...,23.110001,23.110001,23.110001,23.110001,21.173401,100.0,...,NaN,23.371867,23.197681,-0.174186,23.293398,510900.0,2022-04-05,0.638688,0.318171,0.999137


# Using textblob to add more sentiment score

In [9]:
# trying textblob since none of those plots look statistically significant
from textblob import TextBlob
#function to calculate sentiment polarity
def calculate_sentiment(text):
    return TextBlob(text).sentiment.polarity
#apply the function 
df_news['sentiment'] = df_news['Article Text'].apply(calculate_sentiment)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# optional custom tokenizer for advanced control (use if needed)
def process_text(text):
    tokens = word_tokenize(text)
    return [word for word in tokens if word.isalpha()] 

# apply TF-IDF Vectorizer with stopword removal within the vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english') 
tfidf_matrix = tfidf_vectorizer.fit_transform(df_news['Article Text'])
feature_names = tfidf_vectorizer.get_feature_names_out()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aratafujii/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aratafujii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from sklearn.preprocessing import MinMaxScaler

# run minmax scaler on the sentiment scores
scaler = MinMaxScaler(feature_range=(-1, 1))
scores = ["Sentiment Scorecardiffnlp/twitter-roberta-base-sentiment-latest", "Sentiment Scorenlptown/bert-base-multilingual-uncased-sentiment", "Sentiment Scorehakonmh/sentiment-xdistil-uncased", "sentiment"]
new_score_names = ['Twitter Bert Sentiment', 'Bert Multilingual Sentiment', 'Xdistil Uncased Sentiment', 'Text Blob Sentiment']
df_news.rename(columns=dict(zip(scores, new_score_names)), inplace=True)
scores = new_score_names
for score in scores:
    df_news[score] = scaler.fit_transform(df_news[[score]].values)
df_news.head()

,Title,Intro Text,Authors,Article Text,Open,High,Low,Close,ESGB,Volume,...,SUSBBackAvg_14,SUSBForwAvg_14,SUSBDifference_14,SUSBClose_14,SUSBVolume_14,Date,Twitter Bert Sentiment,Bert Multilingual Sentiment,Xdistil Uncased Sentiment,Text Blob Sentiment
0,Liz Truss ‘will approve more oil drilling if ...,Tory leadership candidate criticised by campai...,"['Rob Davies', '@ByRobDavies']",Liz Truss will sign off on a push for more oil...,21.813000,21.813000,21.813000,21.813000,20.171234,0.0,...,23.110091,22.889202,-0.220888,22.994223,114600.0,2022-08-30,0.288852,-0.732887,0.597408,-0.451626
2,Visiting green spaces deters mental health dr...,Positive effects were stronger among those rep...,"['Damien Gayle', '@damiengayle']","Visits to parks, community gardens and other u...",21.299999,21.320000,21.299999,21.309999,20.028095,4000.0,...,23.013015,23.240937,0.227922,23.165962,194600.0,2023-01-17,0.543548,-0.474818,-0.603058,-0.581151
3,Bought too much red cabbage? Turn it into a fe...,This fantastic vegan centrepiece makes full us...,['Tom Hunt'],"I devised today’s nut roast for Oddbox, a veg ...",21.110001,21.110001,21.084999,21.084999,20.707699,2400.0,...,24.064723,24.244775,0.180052,24.224943,89900.0,2023-12-22,0.170077,0.097707,0.994401,-0.304792
4,‘This year has been very good’: readers’ UK bu...,Readers share their favourite sightings over t...,['Guardian readers'],‘Constant companions to our gardening’A peacoc...,21.000000,21.000000,21.000000,21.000000,20.624222,100.0,...,23.998831,24.218568,0.219737,24.145615,76700.0,2023-12-19,-0.526651,-0.018047,0.968052,-0.286398
5,"Country diary: Willow tits are here, but good ...","Tittesworth Reservoir, Staffordshire: It’s eas...",['Mark Cocker'],The numerous bird feeders here are permanently...,23.110001,23.110001,23.110001,23.110001,21.173401,100.0,...,23.371867,23.197681,-0.174186,23.293398,510900.0,2022-04-05,-0.121053,-0.725642,0.998021,-0.372089


# Categorization
trying to break down the data to see if trends exists in smaller groups

In [12]:
from collections import defaultdict

target_words = {'government', 'water', 'climate', 'change', 'energy', 'emission', 'gas', 
                'carbon', 'environment', 'food', 'crisis', 'oil', 'air', 'sea', 'plastic', 'pollution',
                'coal', 'temperature', 'wildlife', 'forest', 'fire', 'policy', 'conservation', 'oceans', 'climate',
                'recycling','waste', 'deforestation', 'renewable'}

from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

def categorize_article(text):
    # preprocess words
    words = text.lower().split()
    words = [word.strip(',.') for word in words if word not in stopwords]

    categories = set()

    # check if the article contains any target words
    for word in words:
        if word in target_words:
            categories.add(word)
    
    # if no target word is found, add 'other' to categories
    if not categories:
        categories.add('other')
    
    return categories

# Add a new column 'Categories' to df_news based on the article text
df_news['Category'] = df_news['Article Text'].dropna().apply(categorize_article)

# Example usage:
print(df_news[['Article Text', 'Category']].head(10))

                                         Article Text  \
0   Liz Truss will sign off on a push for more oil...   
2   Visits to parks, community gardens and other u...   
3   I devised today’s nut roast for Oddbox, a veg ...   
4   ‘Constant companions to our gardening’A peacoc...   
5   The numerous bird feeders here are permanently...   
7   The climate crisis could lead to more small-bo...   
8   Energy companies want the government to implem...   
11  I’m lying on my front on the moist, mossy lawn...   
12  Good morning. Last year, the Cop26 climate sum...   
13  Consumers could be hit with higher egg prices ...   

                                             Category  
0    {policy, gas, change, climate, sea, oil, energy}  
2                                       {policy, sea}  
3                                        {oil, water}  
4                      {wildlife, food, conservation}  
5                                       {waste, coal}  
7                {crisis, temperatur

# Clean up and exporting

In [13]:
df_news['ESGB Difference 7'] = df_news['ESGBDifference']
df_news['SUSB Difference 7'] = df_news['SUSBDifference']
df_news['ESGB Difference 14'] = df_news['ESGBDifference_14']
df_news['SUSB Difference 14'] = df_news['SUSBDifference_14']

In [14]:
df_main = df_news[['Date', 'Article Text', 'Category','Twitter Bert Sentiment', 'Bert Multilingual Sentiment', 'Xdistil Uncased Sentiment', 'Text Blob Sentiment', 'ESGB Difference 7', 'SUSB Difference 7', 'ESGB Difference 14', 'SUSB Difference 14']]
df_main.reset_index(inplace = True)
df_main.describe()

,index,Date,Twitter Bert Sentiment,Bert Multilingual Sentiment,Xdistil Uncased Sentiment,Text Blob Sentiment,ESGB Difference 7,SUSB Difference 7,ESGB Difference 14,SUSB Difference 14
count,11634.000000,11634,11634.000000,11634.000000,11634.000000,11634.000000,11634.000000,11634.000000,11634.000000,11634.000000
mean,9419.900894,2022-10-21 01:06:57.741103872,0.000608,-0.585962,0.713553,-0.401870,-0.014331,0.003769,-0.035366,0.000600
min,0.000000,2021-06-29 00:00:00,-1.000000,-1.000000,-1.000000,-1.000000,-0.504517,-0.327991,-0.621234,-0.349789
25%,4403.500000,2022-03-14 00:00:00,-0.332336,-0.788974,0.604936,-0.458662,-0.144340,-0.053552,-0.254459,-0.093538
50%,8848.500000,2022-10-19 00:00:00,-0.010613,-0.637147,0.922467,-0.403787,-0.023743,0.000138,-0.043829,-0.020854
75%,13219.500000,2023-06-05 00:00:00,0.321786,-0.431438,0.986705,-0.347899,0.092777,0.052945,0.105466,0.086662
max,30053.000000,2024-01-18 00:00:00,1.000000,1.000000,1.000000,1.000000,0.424953,0.285264,0.609189,0.341391
std,6358.168412,NaN,0.391588,0.262175,0.409940,0.092864,0.173429,0.093684,0.289196,0.156708


In [15]:
df_main = df_main.drop(columns = 'index')
df_main.head()

,Date,Article Text,Category,Twitter Bert Sentiment,Bert Multilingual Sentiment,Xdistil Uncased Sentiment,Text Blob Sentiment,ESGB Difference 7,SUSB Difference 7,ESGB Difference 14,SUSB Difference 14
0,2022-08-30,Liz Truss will sign off on a push for more oil...,"{policy, gas, change, climate, sea, oil, energy}",0.288852,-0.732887,0.597408,-0.451626,-0.245126,-0.113282,-0.476402,-0.220888
1,2023-01-17,"Visits to parks, community gardens and other u...","{policy, sea}",0.543548,-0.474818,-0.603058,-0.581151,0.189310,0.108340,0.451390,0.227922
2,2023-12-22,"I devised today’s nut roast for Oddbox, a veg ...","{oil, water}",0.170077,0.097707,0.994401,-0.304792,0.103670,0.070829,0.261671,0.180052
3,2023-12-19,‘Constant companions to our gardening’A peacoc...,"{wildlife, food, conservation}",-0.526651,-0.018047,0.968052,-0.286398,0.273184,0.145756,0.399931,0.219737
4,2022-04-05,The numerous bird feeders here are permanently...,"{waste, coal}",-0.121053,-0.725642,0.998021,-0.372089,-0.306366,-0.089482,-0.496644,-0.174186


In [16]:
#export as csv
df_main.to_csv(path_or_buf='/Users/aratafujii/Desktop/Project2/main_data.csv', index = False)